In [37]:
import re
import numpy as np
import pandas as pd
import contractions
import inflect
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk import WordNetLemmatizer
from bs4 import BeautifulSoup

import pandas as pd
import contractions
import re
from word2number import w2n
import nltk
#nltk.download()
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer
from tqdm import tqdm
from tqdm.auto import tqdm
tqdm.pandas()

from sklearn.preprocessing import MinMaxScaler
from math import log
import json
import matplotlib.pyplot as plt
import pickle

import os
import seaborn as sns
import scipy
import random
from datetime import datetime
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

In [81]:
with open('../movies_train.pkl', 'rb') as f:
    train_set = pickle.load(f)
with open('../movies_test.pkl', 'rb') as f:
    test_set = pickle.load(f)
with open('../movies_validation.pkl', 'rb') as f:
    valid_set = pickle.load(f)

## Example link
- https://github.com/PreferredAI/recommendation-retrieval/blob/37da288631267e8099ca60b94d172c5baa5be400/Part-1-Matrix-Factorization-Recommendation-Retrieval.ipynb


1. 일단 ProbabilisticeMatrixFactorization.py 파일의 fit 부분 잘 봐
2. 보면, train, test data input으로 넣게 되어 있음.
3. 난 이미 train, test 나뉘어져 있기 때문에 이걸 여기서 원하는 형태로 잘 만들어놔줘
4. 이 때, sentiment train, test  //  rating train, test 총 4개 준비해둬.
5. fit 함수의 self.mean_inv 보면 train_vec의 rating 부분 활용하고 있는거 볼 수 있음.
6. 이 부분을 잘 수정해서 SBMF 만들어야해. 
7. fit 안에 rmse도 있는데, mae도 추가해야해.
8. sentiment를 어떻게 녹여놓는가가 관건


# 데이터 준비

In [90]:
train_data = train_set.append(valid_set)
train_data

,userID,movieID,rating,compound,senti_score,NeMFRating,SBMFRating,anger,anticipation,disgust,fear,joy,sadness,surprise,trust,negative,positive
1633870,143756,31800,5.0,0.5719,3,3,4,0.00,0.00,0.00,0.00,16.67,0.00,0.00,33.33,0.00,16.67
3115182,272600,13746,5.0,0.0000,2,3,3,0.00,11.11,0.00,11.11,0.00,0.00,0.00,11.11,0.00,0.00
2383131,208589,23849,3.0,-0.2023,1,2,3,10.53,5.26,5.26,5.26,0.00,5.26,5.26,5.26,10.53,10.53
2079042,182422,14736,4.0,0.9666,3,3,5,5.04,4.20,9.24,8.40,6.72,7.56,5.04,4.20,14.29,13.45
651398,56973,53386,5.0,0.9260,3,3,5,0.00,27.27,0.00,4.55,18.18,0.00,13.64,40.91,0.00,27.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2318060,202972,31547,4.0,0.4404,3,3,4,0.00,3.57,0.00,0.00,3.57,0.00,3.57,7.14,0.00,3.57
2103736,184594,53967,5.0,0.9277,3,3,5,0.00,6.67,0.00,6.67,20.00,0.00,6.67,0.00,0.00,26.67
505244,43893,26640,4.0,0.9531,3,3,5,2.17,6.52,1.09,0.00,6.52,1.09,3.26,5.43,10.87,9.78
906648,79756,15221,5.0,0.6369,3,3,4,0.00,0.00,0.00,0.00,50.00,0.00,0.00,50.00,0.00,50.00


In [91]:
# # SBMFRating 타입 float으로 바꾸기
# train_data = train_data.astype({'SBMFRating':'float'})
# test_set = test_set.astype({'SBMFRating':'float'})

In [92]:
train_s = train_data[['userID', 'movieID', 'SBMFRating']]
test_s = test_set[['userID', 'movieID', 'SBMFRating']]


train_pmf = train_data[['userID', 'movieID', 'rating']]
test_pmf = test_set[['userID', 'movieID', 'rating']]

In [93]:
def load_rating_data(df):

    prefer = []

    ls = df.values.tolist()
    for line in ls:  # 打开指定文件
        (userID, movieID, rating) = line  # 数据集中每行有4项
        uid = int(userID)
        mid = int(movieID)
        rat = float(rating)
        prefer.append([uid, mid, rat])
    data = np.array(prefer)
    return data

In [94]:
trains = load_rating_data(train_s)
tests = load_rating_data(test_s)
trainpmf = load_rating_data(train_pmf)
testpmf = load_rating_data(test_pmf)

In [97]:
# 왜...이런거죠
trainpmf

array([[1.43756e+05, 3.18000e+04, 5.00000e+00],
       [2.72600e+05, 1.37460e+04, 5.00000e+00],
       [2.08589e+05, 2.38490e+04, 3.00000e+00],
       ...,
       [4.38930e+04, 2.66400e+04, 4.00000e+00],
       [7.97560e+04, 1.52210e+04, 5.00000e+00],
       [2.11517e+05, 5.28360e+04, 5.00000e+00]])

# sbmf

In [137]:
class PMF(object):
    def __init__(self, num_feat=10, epsilon=1, _lambda=0.1, momentum=0.8, maxepoch=20, num_batches=10, batch_size=1000):
        self.num_feat = num_feat  # Number of latent features,
        self.epsilon = epsilon  # learning rate,
        self._lambda = _lambda  # L2 regularization,
        self.momentum = momentum  # momentum of the gradient,
        self.maxepoch = maxepoch  # Number of epoch before stop,
        self.num_batches = num_batches  # Number of batches in each epoch (for SGD optimization),
        self.batch_size = batch_size  # Number of training samples used in each batches (for SGD optimization)

        self.w_Item = None  # Item feature vectors
        self.w_User = None  # User feature vectors

        self.rmse_train = []
        self.rmse_test = []

        self.mae_train = []
        self.mae_test = []

    # ***Fit the model with train_tuple and evaluate RMSE on both train and test data.  ***********#
    # ***************** train_vec=TrainData, test_vec=TestData*************#
    def fit(self, train_vec, test_vec):

        # mean subtraction
        self.mean_inv = np.mean(train_vec[:, 2])  # 评分平均值

        pairs_train = train_vec.shape[0]  # traindata 中条目数
        pairs_test = test_vec.shape[0]  # testdata中条目数

        # 1-p-i, 2-m-c
        num_user = int(max(np.amax(train_vec[:, 0]), np.amax(test_vec[:, 0]))) + 1  # 第0列，user总数
        num_item = int(max(np.amax(train_vec[:, 1]), np.amax(test_vec[:, 1]))) + 1  # 第1列，movie总数

        incremental = False  # 增量
        if ((not incremental) or (self.w_Item is None)):
            # initialize
            self.epoch = 0
            self.w_Item = 0.1 * np.random.randn(num_item, self.num_feat)  # numpy.random.randn 电影 M x D 正态分布矩阵
            self.w_User = 0.1 * np.random.randn(num_user, self.num_feat)  # numpy.random.randn 用户 N x D 正态分布矩阵

            self.w_Item_inc = np.zeros((num_item, self.num_feat))  # 创建电影 M x D 0矩阵
            self.w_User_inc = np.zeros((num_user, self.num_feat))  # 创建用户 N x D 0矩阵

        while self.epoch < self.maxepoch:  # 检查迭代次数
            self.epoch += 1

            # Shuffle training truples
            shuffled_order = np.arange(train_vec.shape[0])  # 根据记录数创建等差array
            np.random.shuffle(shuffled_order)  # 用于将一个列表中的元素打乱

            # Batch update
            for batch in range(self.num_batches):  # 每次迭代要使用的数据量
                # print "epoch %d batch %d" % (self.epoch, batch+1)

                test = np.arange(self.batch_size * batch, self.batch_size * (batch + 1))
                batch_idx = np.mod(test, shuffled_order.shape[0])  # 本次迭代要使用的索引下标

                batch_UserID = np.array(train_vec[shuffled_order[batch_idx], 0], dtype='int32')
                batch_ItemID = np.array(train_vec[shuffled_order[batch_idx], 1], dtype='int32')

                # Compute Objective Function
                pred_out = np.sum(np.multiply(self.w_User[batch_UserID, :],
                                              self.w_Item[batch_ItemID, :]),
                                  axis=1)  # mean_inv subtracted # np.multiply对应位置元素相乘

                rawErr = pred_out - train_vec[shuffled_order[batch_idx], 2] + self.mean_inv

                # Compute gradients
                Ix_User = 2 * np.multiply(rawErr[:, np.newaxis], self.w_Item[batch_ItemID, :]) \
                       + self._lambda * self.w_User[batch_UserID, :]
                Ix_Item = 2 * np.multiply(rawErr[:, np.newaxis], self.w_User[batch_UserID, :]) \
                       + self._lambda * (self.w_Item[batch_ItemID, :])  # np.newaxis :increase the dimension

                dw_Item = np.zeros((num_item, self.num_feat))
                dw_User = np.zeros((num_user, self.num_feat))

                # loop to aggreate the gradients of the same element
                for i in range(self.batch_size):
                    dw_Item[batch_ItemID[i], :] += Ix_Item[i, :]
                    dw_User[batch_UserID[i], :] += Ix_User[i, :]

                # Update with momentum
                self.w_Item_inc = self.momentum * self.w_Item_inc + self.epsilon * dw_Item / self.batch_size
                self.w_User_inc = self.momentum * self.w_User_inc + self.epsilon * dw_User / self.batch_size

                self.w_Item = self.w_Item - self.w_Item_inc
                self.w_User = self.w_User - self.w_User_inc

                # Compute Objective Function after
                if batch == self.num_batches - 1:
                    pred_out = np.sum(np.multiply(self.w_User[np.array(train_vec[:, 0], dtype='int32'), :],
                                                  self.w_Item[np.array(train_vec[:, 1], dtype='int32'), :]),
                                      axis=1)  # mean_inv subtracted
                    rawErr = pred_out - train_vec[:, 2] + self.mean_inv
                    obj = np.linalg.norm(rawErr) ** 2 \
                          + 0.5 * self._lambda * (np.linalg.norm(self.w_User) ** 2 + np.linalg.norm(self.w_Item) ** 2)

                        

                    self.rmse_train.append(np.sqrt(obj / pairs_train))

                # Compute validation error
                if batch == self.num_batches - 1:
                    pred_out = np.sum(np.multiply(self.w_User[np.array(test_vec[:, 0], dtype='int32'), :],
                                                  self.w_Item[np.array(test_vec[:, 1], dtype='int32'), :]),
                                      axis=1)  # mean_inv subtracted
                    rawErr = pred_out - test_vec[:, 2] + self.mean_inv
                    self.rmse_test.append(np.linalg.norm(rawErr) / np.sqrt(pairs_test))

                    # Print info
                    if batch == self.num_batches - 1:
                        print('Training RMSE: %f, Test RMSE %f' % (self.rmse_train[-1], self.rmse_test[-1]))
        
        return self.rmse_train, self.rmse_test


    def predict(self, invID):
        return np.dot(self.w_Item, self.w_User[int(invID), :]) + self.mean_inv  # numpy.dot 点乘

    # ****************Set parameters by providing a parameter dictionary.  ***********#
    def set_params(self, parameters):
        if isinstance(parameters, dict):
            self.num_feat = parameters.get("num_feat", 10)
            self.epsilon = parameters.get("epsilon", 1)
            self._lambda = parameters.get("_lambda", 0.1)
            self.momentum = parameters.get("momentum", 0.8)
            self.maxepoch = parameters.get("maxepoch", 20)
            self.num_batches = parameters.get("num_batches", 10)
            self.batch_size = parameters.get("batch_size", 1000)

    def topK(self, test_vec, k=10):
        inv_lst = np.unique(test_vec[:, 0])
        pred = {}
        for inv in inv_lst:
            if pred.get(inv, None) is None:
                pred[inv] = np.argsort(self.predict(inv))[-k:]  # numpy.argsort索引排序

        intersection_cnt = {}
        for i in range(test_vec.shape[0]):
            if test_vec[i, 1] in pred[test_vec[i, 0]]:
                intersection_cnt[test_vec[i, 0]] = intersection_cnt.get(test_vec[i, 0], 0) + 1
        invPairs_cnt = np.bincount(np.array(test_vec[:, 0], dtype='int32'))

        precision_acc = 0.0
        recall_acc = 0.0
        for inv in inv_lst:
            precision_acc += intersection_cnt.get(inv, 0) / float(k)
            recall_acc += intersection_cnt.get(inv, 0) / float(invPairs_cnt[int(inv)])

        return precision_acc / len(inv_lst), recall_acc / len(inv_lst)


In [142]:
pmf = PMF()

In [143]:
rmse_train1, rmse_test1 = pmf.fit(trains, tests)

Training RMSE: 1.050220, Test RMSE 1.051084
Training RMSE: 1.050211, Test RMSE 1.051083
Training RMSE: 1.050202, Test RMSE 1.051082
Training RMSE: 1.050192, Test RMSE 1.051081
Training RMSE: 1.050183, Test RMSE 1.051079
Training RMSE: 1.050173, Test RMSE 1.051079
Training RMSE: 1.050164, Test RMSE 1.051078
Training RMSE: 1.050155, Test RMSE 1.051077
Training RMSE: 1.050146, Test RMSE 1.051075
Training RMSE: 1.050138, Test RMSE 1.051072
Training RMSE: 1.050129, Test RMSE 1.051070
Training RMSE: 1.050120, Test RMSE 1.051069
Training RMSE: 1.050111, Test RMSE 1.051068
Training RMSE: 1.050102, Test RMSE 1.051066
Training RMSE: 1.050094, Test RMSE 1.051065
Training RMSE: 1.050085, Test RMSE 1.051064
Training RMSE: 1.050076, Test RMSE 1.051064
Training RMSE: 1.050068, Test RMSE 1.051064
Training RMSE: 1.050059, Test RMSE 1.051063
Training RMSE: 1.050051, Test RMSE 1.051062


In [145]:
len(rmse_train1)

20

In [114]:
def sbmf(train, test, train_s, test_s):
     pmf = PMF()
     pmf_origin = pmf.fit(train, test)
     pmf_senti = pmf.fit(train_s, test_s)
     sbmf = 0.5*pmf_origin + 0.5*pmf_senti

# SBMF(with surprise package)
- 쉽게가면.... svd(bias=FALSE) 해주면 됨.

In [146]:
from surprise import SVD, KNNBasic, KNNWithMeans, KNNBaseline, NMF
from surprise import dump
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

from collections import defaultdict

In [148]:
with open('../movies_train.pkl', 'rb') as f:
    train_set = pickle.load(f)
with open('../movies_test.pkl', 'rb') as f:
    test_set = pickle.load(f)
with open('../movies_validation.pkl', 'rb') as f:
    valid_set = pickle.load(f)

In [149]:
train_data = train_set.append(valid_set)
train_data

,userID,movieID,rating,compound,senti_score,NeMFRating,SBMFRating,anger,anticipation,disgust,fear,joy,sadness,surprise,trust,negative,positive
1633870,143756,31800,5.0,0.5719,3,3,4,0.00,0.00,0.00,0.00,16.67,0.00,0.00,33.33,0.00,16.67
3115182,272600,13746,5.0,0.0000,2,3,3,0.00,11.11,0.00,11.11,0.00,0.00,0.00,11.11,0.00,0.00
2383131,208589,23849,3.0,-0.2023,1,2,3,10.53,5.26,5.26,5.26,0.00,5.26,5.26,5.26,10.53,10.53
2079042,182422,14736,4.0,0.9666,3,3,5,5.04,4.20,9.24,8.40,6.72,7.56,5.04,4.20,14.29,13.45
651398,56973,53386,5.0,0.9260,3,3,5,0.00,27.27,0.00,4.55,18.18,0.00,13.64,40.91,0.00,27.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2318060,202972,31547,4.0,0.4404,3,3,4,0.00,3.57,0.00,0.00,3.57,0.00,3.57,7.14,0.00,3.57
2103736,184594,53967,5.0,0.9277,3,3,5,0.00,6.67,0.00,6.67,20.00,0.00,6.67,0.00,0.00,26.67
505244,43893,26640,4.0,0.9531,3,3,5,2.17,6.52,1.09,0.00,6.52,1.09,3.26,5.43,10.87,9.78
906648,79756,15221,5.0,0.6369,3,3,4,0.00,0.00,0.00,0.00,50.00,0.00,0.00,50.00,0.00,50.00


In [150]:
train_s = train_data[['userID', 'movieID', 'SBMFRating']]
test_s = test_set[['userID', 'movieID', 'SBMFRating']]


train_pmf = train_data[['userID', 'movieID', 'rating']]
test_pmf = test_set[['userID', 'movieID', 'rating']]

In [151]:
seed_num = 42

In [152]:
# 패키지 쓰려면 이거 설정해줘야함.
reader = Reader(rating_scale=(1, 5))
# train, test set 완성
trainset = Dataset.load_from_df(train_data[['userID', 'movieID', 'rating']], reader)
trainset = trainset.build_full_trainset()

testset = Dataset.load_from_df(test_set[['userID', 'movieID', 'rating']], reader)
testset = testset.construct_testset(testset.raw_ratings)


train_s = Dataset.load_from_df(train_data[['userID', 'movieID', 'SBMFRating']], reader)
train_s = train_s.build_full_trainset()


test_s = Dataset.load_from_df(test_set[['userID', 'movieID', 'SBMFRating']], reader)
test_s = test_s.construct_testset(test_s.raw_ratings)


In [172]:
# number of k size
k = [i for i in range(5,26,5)]

In [167]:
t = [5,10]

In [159]:
# 패키지 사용
def pmf(train, test, i):
     mae_pmf = []
     rmse_pmf = []

     algo = SVD(biased=False, n_factors=i, verbose = 0) # The number of iteration of the SGD procedure.
     np.random.seed(seed_num)

     algo.fit(train)
     predictions_test = algo.test(test)
     error_mae = accuracy.mae(predictions_test, verbose = 0)  
     error_rmse = accuracy.rmse(predictions_test, verbose = 0)

     mae_pmf.append(error_mae)
     rmse_pmf.append(error_rmse)

     print("K:", i, "MAE:",  error_mae,"RMSE",error_rmse)

     return mae_pmf, rmse_pmf

In [161]:
for i in t:
     mae_pmf, rmse_pmf = pmf(train_s,test_s,i)
     print(mae_pmf)
     print(rmse_pmf)

K: 5 MAE: 0.827064557466473 RMSE 1.1361297486206452
[0.827064557466473]
[1.1361297486206452]
K: 10 MAE: 0.840242351538464 RMSE 1.1461689748498969
[0.840242351538464]
[1.1461689748498969]


In [176]:
def sbmf(train, test, train_s, test_s):
     mae_sbmf = []
     rmse_sbmf = []

     algo = SVD(biased=False, n_factors=i, verbose = 0) # The number of iteration of the SGD procedure.
     np.random.seed(seed_num)

     # 평점으로 pmf
     algo.fit(train)
     predictions_test = algo.test(test)

     predictions = []
     for _, _, _, predicted_rating, _ in predictions_test:
          # predicted_rating = norm_list(origin_predictions)
          predictions.append(predicted_rating)

     # sentiment score로 pmf
     algo.fit(train_s)
     predictions_test_s = algo.test(test_s)

     predictions_senti = []
     for _, _, _, predicted_rating, _ in predictions_test_s:
          # predicted_rating = norm_list(origin_predictions)
          predictions_senti.append(predicted_rating)

     # sbmf 예상 rating
     sbmf_rating= [0.5*(i+j) for i,j in zip(predictions,predictions_senti)]




     # 원래 test data의 평점도 리스트로 만들어줌
     origin_rating = []
     for _, _, rating, _, _ in predictions_test:
          origin_rating.append(rating)

     # 원래 sentiment의 test data의 평점도 리스트로 만들어줌
     senti_rating = []
     for _, _, rating, _, _ in predictions_test_s:
          senti_rating.append(rating)

     # sbmf origin rating
     sbmf_origin_rating= [0.5*(i+j) for i,j in zip(origin_rating,senti_rating)]




     # rmse, mae 구하기
     MAE_temp = mean_absolute_error(sbmf_origin_rating, sbmf_rating)
     RMSE_temp = mean_squared_error(sbmf_origin_rating, sbmf_rating, squared = False)

     mae_sbmf.append(MAE_temp)
     rmse_sbmf.append(RMSE_temp)

     print("K:", i, "MAE:",  MAE_temp,"RMSE:",RMSE_temp)

     return mae_sbmf, rmse_sbmf

In [163]:
mae_sbmf, rmse_sbmf = sbmf(trainset, testset, train_s, test_s)

K: 10 MAE: 0.7146861075283099 RMSE: 0.9846483759583007


In [178]:
for i in k:
     mae_sbmf, rmse_sbmf = sbmf(trainset, testset, train_s, test_s)


K: 5 MAE: 0.7023097184896899 RMSE: 0.9731539160479578
K: 10 MAE: 0.7146861075283099 RMSE: 0.9846483759583007
K: 15 MAE: 0.7201372533592957 RMSE: 0.98984873367562
K: 20 MAE: 0.7293915618918204 RMSE: 0.9985823601221085
K: 25 MAE: 0.7378051637184949 RMSE: 1.0059421695599164
